In [ ]:
from pathlib import Path

# 1. Trouve dynamiquement la racine du projet (contenant .gitignore)
cwd = Path.cwd()
PROJECT_ROOT = next(p for p in (cwd, *cwd.parents) if (p / ".gitignore").exists())

print("PROJECT_ROOT =", PROJECT_ROOT)

In [ ]:
# root_dir déjà défini
# root_dir_5_img = PROJECT_ROOT/"dataset"/"plantvillage"/"data"/"plantvillage_5images"/"segmented"
root_dir_img = PROJECT_ROOT/"dataset"/"plantvillage"/"data"/"plantvillage dataset"/"segmented"

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.measure import label, regionprops
from skimage.measure import moments_hu
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.stats import spearmanr
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from collections import defaultdict
import shutil
import hashlib

In [ ]:
df_raw_data = pd.read_csv("Plant_V_Seg_clean.csv")
df_raw_data.columns

In [ ]:
df_raw_data.head()

In [ ]:
df_raw_data.info()

Définitions des fonctions qui extraient des caractéristiques

Les caractéristiques extraites pour chaque image sont :
Caractéristiques de forme
Caractéristiques de couleur
Caractéristiques de texture
Caractéristiques de Densité de contours
Caractéristiques des Moments de Hu

In [ ]:
# Fonction 1 : Caractéristiques de forme 
def extract_shape_features(gray_img, binary_thresh=127):
    """
    Extrait les principales caractéristiques de forme à partir d'une image
    en niveaux de gris ou binaire.
    
    Params:
    - gray_img : np.ndarray, image cv2 (grayscale ou BGR)
    - binary_thresh : int, seuil pour la binarisation si l'image n'est pas déjà binaire
    
    Retourne un dict avec :
    - dimensions (w x h en pixels)
    - aire
    - périmètre
    - circularité
    - excentricité
    - aspect_ratio
    """
    # 1. Si couleur, conversion en niveaux de gris
    if gray_img.ndim == 3:
        gray = cv2.cvtColor(gray_img, cv2.COLOR_BGR2GRAY)
    else:
        gray = gray_img.copy()
    
    # 2. Binarisation (si image non binaire)
    _, binary = cv2.threshold(gray, binary_thresh, 255, cv2.THRESH_BINARY)
    
    # 3. Recherche des contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    
    # 4. Plus grand contour (supposé être la feuille)
    cnt = max(contours, key=cv2.contourArea)
    
    # 5. Calcul de l'aire et du périmètre
    aire = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    
    # 6. Circularité : 4π·A / P²
    circularity = (4 * np.pi * aire) / (perimeter**2) if perimeter > 0 else 0
    
    # 7. Boîte englobante et aspect ratio
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = float(w) / float(h) if h > 0 else 0
    
    # 8. Excentricité via regionprops (sur masque labellisé)
    lbl = label(binary > 0)  # étiquette des composantes
    props = regionprops(lbl)
    if props:
        # on prend la région la plus large (en surface) pour la feuille
        largest_region = max(props, key=lambda p: p.area)
        eccentricity = largest_region.eccentricity
    else:
        eccentricity = 0
    
    # 9. Résultat
    return {
        "dimensions": f"{w}x{h}",
        "aire": float(aire),
        "périmètre": float(perimeter),
        "circularité": float(circularity),
        "excentricité": float(eccentricity),
        "aspect_ratio": float(aspect_ratio)
    }

In [ ]:
# Fonction 2 : Caractéristiques de couleur RGB (moyennes par image) car charge trop importante le PC  plante
def extract_color_features(rgb_img):
    R, G, B = rgb_img[:, :, 0], rgb_img[:, :, 1], rgb_img[:, :, 2]
    return {
        "mean_R": np.mean(R), "mean_G": np.mean(G), "mean_B": np.mean(B),
        "std_R": np.std(R), "std_G": np.std(G), "std_B": np.std(B)
    }

In [ ]:
# Fonction 3 : Texture via GLCM - (Matrice de Co-occurrence de Niveaux de Gris)
def extract_texture_features(gray_img):
    glcm = graycomatrix(gray_img, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    return {
        "contrast": graycoprops(glcm, 'contrast')[0, 0],
        "energy": graycoprops(glcm, 'energy')[0, 0],
        "homogeneity": graycoprops(glcm, 'homogeneity')[0, 0],
        "dissimilarite": graycoprops(glcm, 'dissimilarity')[0, 0],
        "Correlation": graycoprops(glcm, 'correlation')[0, 0],
    }


In [ ]:
# === Fonction 4 : Densité de contours ===
def extract_contour_density(gray_img):
    edges = cv2.Canny(gray_img, 100, 200)
    return {"contour_density": np.sum(edges > 0) / gray_img.size}

In [ ]:
# === Fonction 5 : Moments de Hu ===
def extract_hu_moments(gray_img):
    moments = cv2.moments(gray_img)
    hu = cv2.HuMoments(moments).flatten()
    hu_log = -np.sign(hu) * np.log10(np.abs(hu) + 1e-10)
    return {"hu_moment": hu_log.tolist()}

In [ ]:
# Fonction 6 - 
def extract_hsv_features(rgb):
    """
    Extrait les moyennes des composantes HSV (Hue, Saturation, Value)
    pour une image donnée.

    Paramètre :
        image_path (str) : chemin vers l'image

    Retour :
        dict : {
            "mean_H": float,  # moyenne de la teinte (0–179)
            "mean_S": float,  # moyenne de la saturation (0–255)
            "mean_V": float   # moyenne de la valeur/luminosité (0–255)
        }
    """

    # 1. Convertir en espace HSV
    hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)

    # 2. Séparer les canaux
    h, s, v = cv2.split(hsv)

    # 3. Calculer les moyennes
    return {
        "mean_H": float(np.mean(h)),
        "mean_S": float(np.mean(s)),
        "mean_V": float(np.mean(v))
    }

In [ ]:
# Fonction 7 - Calcule la netteté d'une image via la variance du Laplacien sur l'image convertie en niveaux de gris.
def extract_sharpness_laplacian(image_path):
    """
    Calcule la netteté d'une image via la variance du Laplacien sur l'image
    convertie en niveaux de gris.

    Paramètre :
        image_path (str) : chemin vers l'image

    Retour :
        dict : {
            "netteté": float  # variance du Laplacien
        }
    """
    # 1. Charger l'image et convertir en niveaux de gris
    img = Image.open(image_path).convert("L")
    gray = np.array(img)

    # 2. Appliquer le filtre Laplacien
    lap = cv2.Laplacian(gray, cv2.CV_64F)

    # 3. Calculer la variance du résultat (mesure de netteté)
    var_laplacian = float(np.var(lap))

    return {"netteté": var_laplacian}


In [ ]:

# Vérifie éventuellement les lignes dupliquées dans le DataFrame final (mêmes features + même label).
def detect_duplicate_rows(df):
    """
    Vérifie les lignes dupliquées dans le DataFrame (mêmes valeurs de colonnes).
    """
    print("\n🔎 Vérification des doublons de lignes dans le DataFrame...")
    duplicated_rows = df[df.duplicated()]
    if not duplicated_rows.empty:
        print(f" {len(duplicated_rows)} lignes dupliquées trouvées.")
        return duplicated_rows
    else:
        print(" Aucun doublon de ligne trouvé.")
        return pd.DataFrame()

In [ ]:
# === Fonction principale d'extraction ===

def extract_all_features(image_path, target_size=(224, 224)):
    try:
        # 1. Chargement et conversion en RGB (format PIL)
        img = Image.open(image_path).convert("RGB")
        img = img.resize(target_size)
        rgb = np.array(img)  # uint8 [0,255]
        
        # 2. Conversion en niveaux de gris pour les fonctions qui en ont besoin
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)  # uint8 [0,255]
        
        # 3. Extraction des features par bloc (aucune normalisation)
        shape = extract_shape_features(gray) or {}
        color = extract_color_features(rgb)
        texture = extract_texture_features(gray)
        contour = extract_contour_density(gray)
        hu = extract_hu_moments(gray)
        features_HSV = extract_hsv_features(rgb)
        nettete = extract_sharpness_laplacian(image_path)

        # 4. Fusion des dictionnaires
        all_features = {**shape, **color, **texture, **contour, **features_HSV, **nettete}
        if hu and "hu_moment" in hu:
            all_features.update({f"hu_{i+1}": hu["hu_moment"][i] for i in range(7)})

        return all_features
    except Exception as e:
        print(f"Erreur sur {image_path}: {e}")
        return None


In [ ]:
def build_feature_dataframe(df_raw_data: pd.DataFrame) -> pd.DataFrame:
    """
    Construit un DataFrame de features à partir de df_raw_data en appelant extract_all_features

    Paramètres
    ----------
    df_raw_data : pd.DataFrame
        DataFrame contenant les colonnes :
        - nom_plante, nom_maladie, Est_Saine, Image_Path, width, height, is_black, md5

    Retour
    ------
    pd.DataFrame
        DataFrame préparé contenant :
        - ID_Image (int)
        - Est_Saine, is_black (int)
        - aspect_ratio (float), aire (int)
        - one-hot encodage pour nom_plante et nom_maladie
        - toutes les features extraites par extract_all_features
    """
    entries = []
    for _, row in df_raw_data.iterrows():
        img_path = row['Image_Path']
        try:
            feats = extract_all_features(img_path)
            if feats is None:
                continue

            entry = {
                'ID_Image': len(entries) + 1,
                'nom_plante': row['nom_plante'],
                'nom_maladie': row['nom_maladie'] if pd.notna(row['nom_maladie']) else 'Aucune',
                'Est_Saine': int(row['Est_Saine']),
                'Image_Path': img_path,
                'width_img': int(row['width_img']),
                'height_img': int(row['height_img']),
                'is_black': int(row['is_black']),
                'md5': row['md5']
            }
            entry.update(feats)
            entries.append(entry)
        except Exception as e:
            print(f"[Erreur] {img_path}: {e}")
            continue

    df = pd.DataFrame(entries)

    # Post-traitement des features
    df['Est_Saine'] = df['Est_Saine'].astype(int)
    df['is_black'] = df['is_black'].astype(int)

    df = pd.get_dummies(
        df,
        columns=['nom_plante', 'nom_maladie'],
        prefix=['plant', 'disease'],
        drop_first=False
    )

    to_drop = ['md5', 'width_img', 'height_img']
    df.drop(columns=[col for col in to_drop if col in df.columns], inplace=True)

    return df


In [ ]:
# === Appliquer à dataset ===
reco_plant = build_feature_dataframe(df_raw_data)
print(reco_plant.head())

In [ ]:
reco_plant.columns

In [ ]:
reco_plant.info()

In [ ]:
missing_ids = reco_plant[reco_plant['aire'].isna()]['ID_Image'].tolist()
print("Pas de contours pour ces images :", missing_ids)

In [ ]:
# 2. Filtrer le DataFrame
missing_df = reco_plant[reco_plant['ID_Image'].isin(missing_ids)]

# 3. Préparer la grille d'affichage (3 colonnes)
n = len(missing_df)
cols = 3
rows = (n + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))

# 4. Parcourir et afficher
for ax, (_, row) in zip(axes.flatten(), missing_df.iterrows()):
    img_id = row['ID_Image']
    path   = row['Image_Path']
    # Charger l'image
    img = cv2.imread(path)
    if img is None:
        ax.text(0.5, 0.5, f"Fichier non trouvé\n{path}", 
                ha='center', va='center', color='red')
    else:
        # Convertir BGR → RGB pour matplotlib
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
    ax.set_title(f"ID_Image = {img_id}")
    ax.axis('off')

# 5. Masquer les subplots vides
for ax in axes.flatten()[n:]:
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Filtrage du DataFrame pour retirer ces lignes
df_clean = reco_plant[~reco_plant['ID_Image'].isin(missing_ids)].reset_index(drop=True)

# 3. (Optionnel) Vérifier qu’ils ont bien disparu
print("Ancien nombre de lignes :", len(reco_plant))
print("Nouveau nombre de lignes :", len(df_clean))


In [ ]:
df_clean.to_csv("Plant_V_Seg_all_features.csv", index=False)